In [2]:
#作成したテンソルを読み込み
import torch
X_train = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_train.pt')
X_test = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_test.pt')
X_valid = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/X_valid.pt')
Y_train = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_train.pt')
Y_test = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_test.pt')
Y_valid = torch.load('/content/drive/MyDrive/Colab Notebooks/chapter08/Y_valid.pt')

In [3]:
# モデルの構築(nnはニューラルネットワーク)
from torch import nn

class SLNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size) #全結合層の作成

    def forward(self, x):
        logits = self.fc(x)                          #フォワードパスの作成
        return logits

#入力300,出力4のサイズでインスタンスを作成
model = SLNet(300, 4)
print(model)

SLNet(
  (fc): Linear(in_features=300, out_features=4, bias=True)
)


In [4]:
# データセットを作成する
import torch.utils.data as data

#データセットのクラスを作成
class NewsDataset(data.Dataset):
    """
    newsのDatasetクラス

    Attributes
    ----------------------------
    X : テンソル
        単語ベクトルの平均をまとめたテンソル
    y : テンソル
        カテゴリをラベル化したテンソル
    phase : 'train' or 'val'
        学習か訓練かを設定する
    """
    def __init__(self, X, y, phase='train'):
        self.X = X
        self.y = y
        self.phase = phase

    def __len__(self):
        """全データサイズを返す"""
        return len(self.y)

    def __getitem__(self, idx):
        """idxに対応するテンソル形式のデータとラベルを取得"""
        return self.X[idx], self.y[idx]

train_dataset = NewsDataset(X_train, Y_train, phase='train')
valid_dataset = NewsDataset(X_valid, Y_valid, phase='val')
test_dataset = NewsDataset(X_test, Y_test, phase='val')

# 動作確認
idx = 0
print(train_dataset.__getitem__(idx)[0].size())
print(train_dataset.__getitem__(idx)[1])
print(valid_dataset.__getitem__(idx)[0].size())
print(valid_dataset.__getitem__(idx)[1])
print(test_dataset.__getitem__(idx)[0].size())
print(test_dataset.__getitem__(idx)[1])

torch.Size([300])
tensor(2)
torch.Size([300])
tensor(3)
torch.Size([300])
tensor(2)


In [5]:
# DataLoaderを作成(Dataloaderはdatasetsからバッチごとに取り出すことを目的とする)
batch_size = 1

train_dataloader = data.DataLoader(                                              #ここには10671個のバッチが含まれている
            train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = data.DataLoader(
            valid_dataset, batch_size=len(valid_dataset), shuffle=False)
test_dataloader = data.DataLoader(
            test_dataset, batch_size=len(test_dataset), shuffle=False)

dataloaders_dict = {'train': train_dataloader,
                    'val': valid_dataloader,
                    'test': test_dataloader,
                   }

# 動作確認
batch_iter = iter(dataloaders_dict['train']) #バッチからデータを取り出す
inputs, labels = next(batch_iter)
print(inputs.size())
print(labels)

torch.Size([1, 300])
tensor([2])


In [6]:
from tqdm import tqdm
# 学習

# モデルの定義
net = SLNet(300, 4)
net.train()

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 最適化手法の定義
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)       #確率的勾配降下法、学習率0.1、モーメンタムは0.9

# 学習用の関数を定義
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {} / {}'.format(epoch + 1, num_epochs))
        print('--------------------------------------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 訓練モード
            else:
                net.eval() # 検証モード

            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                optimizer.zero_grad() # optimizerを初期化

                # 順伝播計算(forward)
                with torch.set_grad_enabled(phase == 'train'):     #訓練モードの時に勾配計算を有効にする
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) # 損失を計算
                    _, preds = torch.max(outputs, 1) # ラベルを予想

                    # 訓練時は逆伝播
                    if phase == 'train':
                        loss.backward()  #誤差逆伝播
                        optimizer.step() #パラメータの更新

                    # イテレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率の表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


# 学習を実行する
num_epochs = 10
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1 / 10
--------------------------------------------


100%|██████████| 10672/10672 [00:09<00:00, 1114.78it/s]


train Loss: 0.4166, Acc: 0.8550


100%|██████████| 1/1 [00:00<00:00, 52.52it/s]


val Loss: 0.3115, Acc: 0.9003
Epoch 2 / 10
--------------------------------------------


...

val Loss: 0.2694, Acc: 0.9085
Epoch 10 / 10
--------------------------------------------


100%|██████████| 10672/10672 [00:08<00:00, 1326.00it/s]


train Loss: 0.2530, Acc: 0.9115


100%|██████████| 1/1 [00:00<00:00, 51.03it/s]

val Loss: 0.2792, Acc: 0.9115


In [7]:
def calc_acc(net, dataloader):
    net.eval()
    corrects = 0
    with torch.no_grad():                                     #テンソルの勾配計算を不可にする
        for inputs, labels in dataloader:                     #データローダからデータとラベルを読み込み
            outputs = net(inputs)                             #inputsをモデルに入力
            _, preds = torch.max(outputs, 1)                  #ラベルを予想
            corrects += torch.sum(preds == labels.data)       #正解数を数える
    return corrects / len(dataloader.dataset)                 #正解率を返す

acc_train = calc_acc(net, train_dataloader)                   #trainデータの精度を計算
acc_valid = calc_acc(net, valid_dataloader)                   #validデータの精度を計算
acc_test = calc_acc(net, test_dataloader)                     #testデータの精度を計算
print('学習データの正解率: {:.4f}'.format(acc_train))
print('検証データの正解率: {:.4f}'.format(acc_valid))
print('テストデータの正解率: {:.4f}'.format(acc_test))

学習データの正解率: 0.9131
検証データの正解率: 0.9115
テストデータの正解率: 0.8913
